In [1]:
import torch
from torch import nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.model_selection import train_test_split

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path = '/content/drive/MyDrive/db.csv'
db = pd.read_csv(file_path, header=None) # Read csv file with no header
db.drop(db.tail(2).index,inplace = True) # Delete 2 last rows as they are totals

In [4]:
scaler = MinMaxScaler()

X = db.iloc[:, 2:] # Get input data
X = scaler.fit_transform(X) # Normalize data
X = torch.from_numpy(X).float()

Y = db.iloc[:, 1] # Get output
Y = Y.replace({'M': 1, 'B': 0}) # 1 means malign, 0 means benign
Y = np.array(Y)
Y = torch.from_numpy(Y).float()

#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, neurons_per_layer, output_size, activation_function):
        super(MLP, self).__init__()
        activation_functions = {
            "sigmoid": nn.Sigmoid(),
            "tanh": nn.Tanh(),
            "relu": nn.ReLU()
        }

        if activation_function.lower() not in activation_functions:
            raise ValueError(f"Unsupported activation function: {activation_function}")

        self.activation_function = activation_functions[activation_function.lower()]

        layers = []
        # Input Layer
        layers.append(nn.Linear(input_size, neurons_per_layer))
        layers.append(self.activation_function)

        # Hidden Layers
        for i in range(hidden_layers):
            layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
            layers.append(self.activation_function)

        # Output Layer
        layers.append(nn.Linear(neurons_per_layer, output_size))

        self.layers = nn.Sequential(*layers)
        self.loss_history = None

    def forward(self, x):
      output = self.layers(x)
      return output.squeeze()

    def fit(self, X_train, Y_train, epochs=3000, learning_rate=0.01):
        loss_function = nn.BCEWithLogitsLoss() #Internally use Softmax function
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        loss_history = []

        for epoch in range(epochs):
          Y_pred = self.forward(X_train)
          loss = loss_function(Y_pred,Y_train)
          loss_history.append(loss.item())
          if (epoch%500==0):
            print(f"Epoch {epoch}: train loss: {loss.item()}")

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

        self.loss_history = loss_history

    def predict(self, X_test):
        self.eval()

        with torch.no_grad():  # Deactive gradient calc for interference
            Y_pred = self.forward(X_test)

        probs = torch.sigmoid(Y_pred)

        labels = probs > 0.5
        return labels.int()  # Return 0 or 1


In [ ]:
X_train, X_test, Y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [10]:
# Configuraciones para los experimentos
hidden_layers_configs = [1, 2, 3]
neurons_per_layer_configs = [16, 32, 64]
activation_functions = ["sigmoid", "tanh", "relu"]

# Diccionario para guardar los resultados
results = {}

for hidden_layers in hidden_layers_configs:
    for neurons in neurons_per_layer_configs:
        for activation_function in activation_functions:
            print(f"{hidden_layers} layers, {neurons} neurons, {activation_function}")
            # Crear y entrenar el modelo
            model = MLP(input_size=X_train.shape[1], hidden_layers=hidden_layers, neurons_per_layer=neurons,
                        output_size=1, activation_function=activation_function)
            model.fit(X_train, Y_train)

            # Evaluar el modelo
            y_pred = model.predict(X_train)
            accuracy = (y_pred == Y_train).float().mean()

            print("------------------------------------------------")
            # Guardar los resultados
            config_key = f"{hidden_layers} layers, {neurons} neurons, {activation_function}"
            results[config_key] = {
                "loss_history": model.loss_history,
                "accuracy": accuracy.item()
            }


1 layers, 16 neurons, sigmoid
Epoch 0: train loss: 0.6875467896461487
Epoch 500: train loss: 0.022613557055592537
Epoch 1000: train loss: 0.0031653502956032753
Epoch 1500: train loss: 0.0009114059503190219
Epoch 2000: train loss: 0.00037857875577174127
Epoch 2500: train loss: 0.00019199847884010524
------------------------------------------------
1 layers, 16 neurons, tanh
Epoch 0: train loss: 0.7128565907478333
Epoch 500: train loss: 0.00857322383671999
Epoch 1000: train loss: 0.0032973478082567453
Epoch 1500: train loss: 0.0014465079875662923
Epoch 2000: train loss: 0.00071050034603104
Epoch 2500: train loss: 0.0003797796671278775
------------------------------------------------
1 layers, 16 neurons, relu
Epoch 0: train loss: 0.6889375448226929
Epoch 500: train loss: 0.08422309905290604
Epoch 1000: train loss: 0.08062631636857986
Epoch 1500: train loss: 0.07105127722024918
Epoch 2000: train loss: 0.06772149354219437
Epoch 2500: train loss: 0.06673706322908401
------------------------

In [34]:
accuracy_dic = {'sigmoid': [], 'relu': [], 'tanh': []}  # Actualiza con tus datos

for config, result in results.items():
    # Extraer la función de activación del nombre de la configuración
    activation_function = config.split(', ')[-1]
    accuracy_dic[activation_function].append(result['accuracy'])



#print(accuracy_dic)

for config, result in results.items():
    # Extraer la función de activación del nombre de la configuración
    activation_function = config.split(', ')[-1]
    if (result['accuracy'] == max(accuracy_dic[activation_function]) and activation_function == 'relu'):
      print(config)
      print('----------------------')
      # plt.plot(result['loss_history'], label=activation_function)
      # plt.title('Función de Pérdida vs Épocas por Función de Activación')
      # plt.xlabel('Épocas')
      # plt.ylabel('Pérdida')
      # plt.legend()
      # plt.show()

# # Crear un gráfico para comparar las funciones de activación
# #plt.figure(figsize=(10, 6))
# accuracy_dic = {'sigmoid': [], 'relu': [], 'tanh': []}


# # Crear el gráfico de líneas
# plt.figure(figsize=(10, 6))

# # Iterar sobre cada función de activación y graficar sus precisiones
# for activation_function, accuracies in accuracy_dic.items():
#     plt.plot(accuracies, label=activation_function)

# plt.title('Valores de Precisión para Cada Función de Activación')
# plt.xlabel('Experimento / Configuración')
# plt.ylabel('Precisión')
# plt.legend()
# plt.show()



1 layers, 32 neurons, relu
----------------------
1 layers, 64 neurons, relu
----------------------
2 layers, 16 neurons, relu
----------------------
2 layers, 32 neurons, relu
----------------------
2 layers, 64 neurons, relu
----------------------
3 layers, 32 neurons, relu
----------------------
3 layers, 64 neurons, relu
----------------------
